In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import glob
import json

import pandas as pd

In [15]:
import os

big_data = pd.DataFrame()
## find current folder root
root = os.getcwd()

all_files = glob.iglob(
    r'..\downloaded_configs\passivbot_v5.9.x\configs/v5.9.3_clock_realistic/clock/toaster_longonly_01_safe/' + '/**/result.txt',
    recursive=True)

files = []
for file in all_files:
    files.append(file)

print(files)

i = 0
for filename in files:
    #print(f'parsing : {filename}')
    json_file = os.path.abspath(filename.replace('result.txt', '../config.json'))
    with open(json_file) as f:
        live_config = json.load(f)
        config_type = live_config['config_name']

    with open(filename, 'r') as file:
        data = file.read().split('\n')
        d = {'file': filename.lstrip('..\\')}
        current_state = 'long'
        for l in data:
            l = l.split('|')
            if len(l) == 4:
                key = l[1].strip()
                value = l[2].strip()

                if key == 'Symbol':
                    d['s'] = value

                if key == 'Long':
                    current_state = 'long'
                    d['long'] = bool(value) or False
                if key == 'Short':
                    current_state = 'short'
                    d['short'] = bool(value) or False

                if key == 'Total gain' and current_state == 'long':
                    d['tgl_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'long':
                    d['adgl_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'long':
                    d['bnkrl_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'long':
                    d['ebrl'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'long':
                    d['maxsl'] = float(value.replace('%', '').strip()) / 24
                if key == 'Price action distance max' and current_state == 'long':
                    d['pamaxl'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'long':
                    d['pastdl'] = float(value.strip())

                if key == 'Total gain' and current_state == 'short':
                    d['tgs_%'] = float(value.replace('%', '').strip())
                if key == 'Average daily gain' and current_state == 'short':
                    d['adgs_%'] = float(value.replace('%', '').strip())
                if key == 'Closest bankruptcy' and current_state == 'short':
                    d['bnkrs_%'] = float(value.replace('%', '').strip())
                if key == 'Lowest equity/balance ratio' and current_state == 'short':
                    d['ebrs'] = float(value.replace('%', '').strip())
                if key == 'Max hours stuck' and current_state == 'short':
                    d['maxss'] = float(value.replace('%', '').strip()) / 24
                if key == 'Price action distance max' and current_state == 'short':
                    d['pamaxs'] = float(value.replace('%', '').strip())
                if key == 'Price action distance std' and current_state == 'short':
                    d['pastds'] = float(value.strip())

        big_data = big_data.append(d, ignore_index=True)
big_data

['..\\downloaded_configs\\passivbot_v5.9.x\\configs/v5.9.3_clock_realistic/clock/toaster_longonly_01_safe\\ADAUSDT_20230217004157_5293c\\ADAUSDT\\result.txt', '..\\downloaded_configs\\passivbot_v5.9.x\\configs/v5.9.3_clock_realistic/clock/toaster_longonly_01_safe\\ALGOUSDT_20230217004157_5293c\\ALGOUSDT\\result.txt', '..\\downloaded_configs\\passivbot_v5.9.x\\configs/v5.9.3_clock_realistic/clock/toaster_longonly_01_safe\\ATOMUSDT_20230217004157_5293c\\ATOMUSDT\\result.txt', '..\\downloaded_configs\\passivbot_v5.9.x\\configs/v5.9.3_clock_realistic/clock/toaster_longonly_01_safe\\AVAXUSDT_20230217004157_5293c\\AVAXUSDT\\result.txt', '..\\downloaded_configs\\passivbot_v5.9.x\\configs/v5.9.3_clock_realistic/clock/toaster_longonly_01_safe\\BNBUSDT_20230217004157_5293c\\BNBUSDT\\result.txt', '..\\downloaded_configs\\passivbot_v5.9.x\\configs/v5.9.3_clock_realistic/clock/toaster_longonly_01_safe\\BTCUSDT_20230217004157_5293c\\BTCUSDT\\result.txt', '..\\downloaded_configs\\passivbot_v5.9.x\\co

C:\Users\skhan\AppData\Local\Temp\ipykernel_26444\1351075178.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp\ipykernel_26444\1351075178.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp\ipykernel_26444\1351075178.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_index=True)
C:\Users\skhan\AppData\Local\Temp\ipykernel_26444\1351075178.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  big_data = big_data.append(d, ignore_inde

,file,s,long,tgl_%,adgl_%,pastdl,pamaxl,bnkrl_%,ebrl,maxsl
0,downloaded_configs\passivbot_v5.9.x\configs/v5...,ADAUSDT,True,259.80,0.1130,0.012441,0.373502,100.0,0.9184,3.354167
1,downloaded_configs\passivbot_v5.9.x\configs/v5...,ALGOUSDT,True,222.60,0.1190,0.017795,0.469379,100.0,0.9042,2.462500
2,downloaded_configs\passivbot_v5.9.x\configs/v5...,ATOMUSDT,True,212.10,0.1020,0.024227,0.989021,100.0,0.8508,2.779863
3,downloaded_configs\passivbot_v5.9.x\configs/v5...,AVAXUSDT,True,168.10,0.1120,0.024155,0.372757,100.0,0.9188,3.810417
4,downloaded_configs\passivbot_v5.9.x\configs/v5...,BNBUSDT,True,129.60,0.0746,0.007881,0.340036,100.0,0.9244,3.586113
5,downloaded_configs\passivbot_v5.9.x\configs/v5...,BTCUSDT,True,80.96,0.0500,0.008509,0.237834,100.0,0.9427,6.844458
6,downloaded_configs\passivbot_v5.9.x\configs/v5...,COMPUSDT,True,107.10,0.0751,0.015357,0.406771,100.0,0.9134,2.728471
7,downloaded_configs\passivbot_v5.9.x\configs/v5...,DOGEUSDT,True,666.40,0.2130,0.010982,0.414025,100.0,0.9122,0.401389
8,downloaded_configs\passivbot_v5.9.x\configs/v5...,DOTUSDT,True,169.00,0.1080,0.019277,0.311392,100.0,0.9289,3.434721
9,downloaded_configs\passivbot_v5.9.x\configs/v5...,DYDXUSDT,True,67.02,0.0973,0.014824,0.217237,100.0,0.9296,3.200000


In [16]:
big_data

,file,s,long,tgl_%,adgl_%,pastdl,pamaxl,bnkrl_%,ebrl,maxsl
0,downloaded_configs\passivbot_v5.9.x\configs/v5...,ADAUSDT,True,259.80,0.1130,0.012441,0.373502,100.0,0.9184,3.354167
1,downloaded_configs\passivbot_v5.9.x\configs/v5...,ALGOUSDT,True,222.60,0.1190,0.017795,0.469379,100.0,0.9042,2.462500
2,downloaded_configs\passivbot_v5.9.x\configs/v5...,ATOMUSDT,True,212.10,0.1020,0.024227,0.989021,100.0,0.8508,2.779863
3,downloaded_configs\passivbot_v5.9.x\configs/v5...,AVAXUSDT,True,168.10,0.1120,0.024155,0.372757,100.0,0.9188,3.810417
4,downloaded_configs\passivbot_v5.9.x\configs/v5...,BNBUSDT,True,129.60,0.0746,0.007881,0.340036,100.0,0.9244,3.586113
5,downloaded_configs\passivbot_v5.9.x\configs/v5...,BTCUSDT,True,80.96,0.0500,0.008509,0.237834,100.0,0.9427,6.844458
6,downloaded_configs\passivbot_v5.9.x\configs/v5...,COMPUSDT,True,107.10,0.0751,0.015357,0.406771,100.0,0.9134,2.728471
7,downloaded_configs\passivbot_v5.9.x\configs/v5...,DOGEUSDT,True,666.40,0.2130,0.010982,0.414025,100.0,0.9122,0.401389
8,downloaded_configs\passivbot_v5.9.x\configs/v5...,DOTUSDT,True,169.00,0.1080,0.019277,0.311392,100.0,0.9289,3.434721
9,downloaded_configs\passivbot_v5.9.x\configs/v5...,DYDXUSDT,True,67.02,0.0973,0.014824,0.217237,100.0,0.9296,3.200000


In [17]:
v5_9_0 = ['OP', 'DYDX', 'XTZ', 'XRP', 'SUSHI', 'SAND', 'MATIC', 'MANA', 'LINK', 'HBAR', 'EOS', 'DOT', 'DOGE', 'CHZ', 'ATOM']
v5_9_3 = ['ADA', 'ALGO', 'ATOM', 'AVAX', 'BNB', 'BTC', 'COMP', 'DOGE', 'DOT', 'DYDX', 'EOS', 'ETC', 'ETH', 'GALA', 'GAL', 'GMT', 'HBAR', 'KSM', 'LINK', 'LTC', 'MANA', 'MATIC', 'NEAR', 'RUNE', 'SAND', 'SUSHI', 'TRX', 'VET', 'XMR', 'XRP', 'XTZ']

common_elements = [elem for elem in v5_9_0 if elem in v5_9_3]

# Append USDT to each element in v5_9_0
v5_9_0 = [s + 'USDT' for s in v5_9_0]

dx = big_data[~big_data['s'].isin(v5_9_0)].reset_index(drop=True)

dx

,file,s,long,tgl_%,adgl_%,pastdl,pamaxl,bnkrl_%,ebrl,maxsl
0,downloaded_configs\passivbot_v5.9.x\configs/v5...,ADAUSDT,True,259.80,0.1130,0.012441,0.373502,100.0,0.9184,3.354167
1,downloaded_configs\passivbot_v5.9.x\configs/v5...,ALGOUSDT,True,222.60,0.1190,0.017795,0.469379,100.0,0.9042,2.462500
2,downloaded_configs\passivbot_v5.9.x\configs/v5...,AVAXUSDT,True,168.10,0.1120,0.024155,0.372757,100.0,0.9188,3.810417
3,downloaded_configs\passivbot_v5.9.x\configs/v5...,BNBUSDT,True,129.60,0.0746,0.007881,0.340036,100.0,0.9244,3.586113
4,downloaded_configs\passivbot_v5.9.x\configs/v5...,BTCUSDT,True,80.96,0.0500,0.008509,0.237834,100.0,0.9427,6.844458
5,downloaded_configs\passivbot_v5.9.x\configs/v5...,COMPUSDT,True,107.10,0.0751,0.015357,0.406771,100.0,0.9134,2.728471
6,downloaded_configs\passivbot_v5.9.x\configs/v5...,ETCUSDT,True,173.40,0.0878,0.010746,0.533994,100.0,0.8954,1.672221
7,downloaded_configs\passivbot_v5.9.x\configs/v5...,ETHUSDT,True,124.10,0.0691,0.014226,0.323470,100.0,0.9266,5.186125
8,downloaded_configs\passivbot_v5.9.x\configs/v5...,GALAUSDT,True,103.10,0.1360,0.021600,0.416372,100.0,0.9118,2.893054
9,downloaded_configs\passivbot_v5.9.x\configs/v5...,GALUSDT,True,83.76,0.2100,0.011031,0.561366,100.0,0.8921,0.277778


In [18]:
import requests


symbols = dx['s'].unique().tolist()
print(symbols)
symbols_with_min_notional = {}
data_url = "https://fapi.binance.com/fapi/v1/exchangeInfo"
data = requests.get(data_url).json()
prices_url = "https://fapi.binance.com/fapi/v1/ticker/price"
pricedata = requests.get(prices_url).json()

# print(data)
# print(pricedata)

try:
    for datas in data["symbols"]:
        if "USDT" in datas["pair"]:
            symbol = datas["pair"]
            #print(f'looking for {symbol} in {symbols}')
            if symbol not in symbols:
                continue
            min_qty = float(datas["filters"][1]["minQty"] )
            min_notional_fixed =  float(datas["filters"][5]["notional"])

            for i in pricedata:
                if i["symbol"] == symbol:
                    price = i["price"]

            min_notional_calc = min_qty * float(price)
            print(f'{symbol.ljust(13)}\t min_notional: ${min_notional_calc:.2f}\t')

            if min_notional_calc <= min_notional_fixed:
                min_notional = min_notional_fixed
            else:
                min_notional = min_notional_calc


            symbols_with_min_notional[symbol] = min_notional
            # if iec >= min_notional:
            #     list.append(f"{symbol.ljust(13)}\t min_notional: ${min_notional:.2f}\t GRID OK")
            # else:
            #     list.append(f"{symbol.ljust(13)}\t min_notional: ${min_notional:.2f}\t GRID NOT OK")

except Exception as e:
    print(f'Error: {e}')
    pass



# function to get minimum trade size
def get_min_trade_size(coin):
    return symbols_with_min_notional.get(coin, 0)

dx['min_size_usdt'] = dx['s'].apply(lambda x: get_min_trade_size(x))

dx

# # Define the symbols for the futures market

#
# print(symbols)
#
# # Loop through the symbols and get the minimum quantity size in USDT for each
# for symbol in symbols:
#     min_qty_size = get_min_qty_size(symbol)
#     print(symbol, min_qty_size)

['ADAUSDT', 'ALGOUSDT', 'AVAXUSDT', 'BNBUSDT', 'BTCUSDT', 'COMPUSDT', 'ETCUSDT', 'ETHUSDT', 'GALAUSDT', 'GALUSDT', 'GMTUSDT', 'KSMUSDT', 'LTCUSDT', 'NEARUSDT', 'RUNEUSDT', 'TRXUSDT', 'VETUSDT', 'XMRUSDT']
Error: string indices must be integers


,file,s,long,tgl_%,adgl_%,pastdl,pamaxl,bnkrl_%,ebrl,maxsl,min_size_usdt
0,downloaded_configs\passivbot_v5.9.x\configs/v5...,ADAUSDT,True,259.80,0.1130,0.012441,0.373502,100.0,0.9184,3.354167,0
1,downloaded_configs\passivbot_v5.9.x\configs/v5...,ALGOUSDT,True,222.60,0.1190,0.017795,0.469379,100.0,0.9042,2.462500,0
2,downloaded_configs\passivbot_v5.9.x\configs/v5...,AVAXUSDT,True,168.10,0.1120,0.024155,0.372757,100.0,0.9188,3.810417,0
3,downloaded_configs\passivbot_v5.9.x\configs/v5...,BNBUSDT,True,129.60,0.0746,0.007881,0.340036,100.0,0.9244,3.586113,0
4,downloaded_configs\passivbot_v5.9.x\configs/v5...,BTCUSDT,True,80.96,0.0500,0.008509,0.237834,100.0,0.9427,6.844458,0
5,downloaded_configs\passivbot_v5.9.x\configs/v5...,COMPUSDT,True,107.10,0.0751,0.015357,0.406771,100.0,0.9134,2.728471,0
6,downloaded_configs\passivbot_v5.9.x\configs/v5...,ETCUSDT,True,173.40,0.0878,0.010746,0.533994,100.0,0.8954,1.672221,0
7,downloaded_configs\passivbot_v5.9.x\configs/v5...,ETHUSDT,True,124.10,0.0691,0.014226,0.323470,100.0,0.9266,5.186125,0
8,downloaded_configs\passivbot_v5.9.x\configs/v5...,GALAUSDT,True,103.10,0.1360,0.021600,0.416372,100.0,0.9118,2.893054,0
9,downloaded_configs\passivbot_v5.9.x\configs/v5...,GALUSDT,True,83.76,0.2100,0.011031,0.561366,100.0,0.8921,0.277778,0


In [23]:

dl_proposed = dx[
    (dx['min_size_usdt'] < 10) &
    #(dx['lm'] == 'n') &
    (dx['bnkrl_%'] == 100) &
    #(dx['bnkrs_%'] == 100) &
    (dx['ebrl'] > 0.9)  #&
    # (dx['ebrs'] > 0.9) &  #&
    # (dx['adgl_%'] > 0.04) 
    # (dx['pastdl'] < 0.03) &
    # (dx['pastds'] < 0.03) &  #
    # (dx['maxsl'] < 10) &
    # (dx['maxss'] < 10) &
    # (dx['adgt_%'].notnull()) &
    # (dx['adgt_%'] > 0)
    ]
pd.set_option('display.max_colwidth', None)
dl_proposed

,file,s,long,tgl_%,adgl_%,pastdl,pamaxl,bnkrl_%,ebrl,maxsl,min_size_usdt
0,downloaded_configs\passivbot_v5.9.x\configs/v5.9.3_clock_realistic/clock/toaster_longonly_01_safe\ADAUSDT_20230217004157_5293c\ADAUSDT\result.txt,ADAUSDT,True,259.80,0.1130,0.012441,0.373502,100.0,0.9184,3.354167,0
1,downloaded_configs\passivbot_v5.9.x\configs/v5.9.3_clock_realistic/clock/toaster_longonly_01_safe\ALGOUSDT_20230217004157_5293c\ALGOUSDT\result.txt,ALGOUSDT,True,222.60,0.1190,0.017795,0.469379,100.0,0.9042,2.462500,0
2,downloaded_configs\passivbot_v5.9.x\configs/v5.9.3_clock_realistic/clock/toaster_longonly_01_safe\AVAXUSDT_20230217004157_5293c\AVAXUSDT\result.txt,AVAXUSDT,True,168.10,0.1120,0.024155,0.372757,100.0,0.9188,3.810417,0
3,downloaded_configs\passivbot_v5.9.x\configs/v5.9.3_clock_realistic/clock/toaster_longonly_01_safe\BNBUSDT_20230217004157_5293c\BNBUSDT\result.txt,BNBUSDT,True,129.60,0.0746,0.007881,0.340036,100.0,0.9244,3.586113,0
4,downloaded_configs\passivbot_v5.9.x\configs/v5.9.3_clock_realistic/clock/toaster_longonly_01_safe\BTCUSDT_20230217004157_5293c\BTCUSDT\result.txt,BTCUSDT,True,80.96,0.0500,0.008509,0.237834,100.0,0.9427,6.844458,0
5,downloaded_configs\passivbot_v5.9.x\configs/v5.9.3_clock_realistic/clock/toaster_longonly_01_safe\COMPUSDT_20230217004157_5293c\COMPUSDT\result.txt,COMPUSDT,True,107.10,0.0751,0.015357,0.406771,100.0,0.9134,2.728471,0
7,downloaded_configs\passivbot_v5.9.x\configs/v5.9.3_clock_realistic/clock/toaster_longonly_01_safe\ETHUSDT_20230217004157_5293c\ETHUSDT\result.txt,ETHUSDT,True,124.10,0.0691,0.014226,0.323470,100.0,0.9266,5.186125,0
8,downloaded_configs\passivbot_v5.9.x\configs/v5.9.3_clock_realistic/clock/toaster_longonly_01_safe\GALAUSDT_20230217004157_5293c\GALAUSDT\result.txt,GALAUSDT,True,103.10,0.1360,0.021600,0.416372,100.0,0.9118,2.893054,0
10,downloaded_configs\passivbot_v5.9.x\configs/v5.9.3_clock_realistic/clock/toaster_longonly_01_safe\GMTUSDT_20230217004157_5293c\GMTUSDT\result.txt,GMTUSDT,True,55.52,0.1300,0.016328,0.127076,100.0,0.9662,2.803471,0
11,downloaded_configs\passivbot_v5.9.x\configs/v5.9.3_clock_realistic/clock/toaster_longonly_01_safe\KSMUSDT_20230217004157_5293c\KSMUSDT\result.txt,KSMUSDT,True,133.70,0.0982,0.007836,0.167660,100.0,0.9202,3.367363,0


In [ ]:
# existing_screens = ['TRX', 'XMR', 'ETC', 'LTC', 'ADA', 'KSM', 'BNB', 'ETH', 'XTZ', 'XRP', 'SUSHI', 'SAND', 'MATIC', 'MANA', 'LINK', 'HBAR', 'EOS', 'DOT', 'DOGE', 'CHZ', 'ATOM']
#
# existing_screens = [x + 'USDT' for x in existing_screens]
# dl_proposed = dl_proposed[~dl_proposed['s'].isin(existing_screens)]
#
# dl_proposed

In [ ]:
# existing_screens = ['TRX', 'XMR', 'ETC', 'LTC', 'ADA', 'KSM', 'BNB', 'ETH', 'XTZ', 'XRP', 'SUSHI', 'SAND', 'MATIC', 'MANA', 'LINK', 'HBAR', 'EOS', 'DOT', 'DOGE', 'CHZ', 'ATOM', 'NEAR']
#
# existing_screens = [x + 'USDT' for x in existing_screens]
# dl_proposed = dl_proposed[~dl_proposed['s'].isin(existing_screens)]
#
# dl_proposed